<a href="https://colab.research.google.com/github/RodrigoCastroAndrade/PythonGeminiAIFirstSteps/blob/main/C%C3%B3pia_de_Imers%C3%A3o_IA_Alura_%2B_Google_Gemini_Aula_05_Agentes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip -q install google-genai

In [2]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [3]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [4]:
# Pergunta ao Gemini uma informação mais recente que seu conhecimento

from IPython.display import HTML, Markdown

# Perguntar pro modelo quando é a próxima imersão de IA ###############################################
resposta = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {resposta.text}"))

Resposta:
 A Alura não tem uma data fixa para a Imersão IA com Google Gemini. A melhor forma de saber quando a próxima edição será realizada é:

*   **Acompanhar a página da Alura:** Fique de olho na página principal da Alura e na seção de cursos de Inteligência Artificial.
*   **Seguir a Alura nas redes sociais:** A Alura costuma divulgar novas edições de seus cursos e imersões em suas redes sociais (LinkedIn, Instagram, etc.).
*   **Assinar a newsletter da Alura:** Ao assinar a newsletter, você receberá informações sobre lançamentos e eventos da Alura diretamente no seu e-mail.

In [5]:
# Pergunta ao Gemini uma informação utilizando a busca do Google como contexto

response = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
    config = {"tools":  [{"google_search": {}}]}
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

Resposta:
 Com base nas informações encontradas, a Alura e o Google realizaram recentemente a "Imersão IA com Google Gemini". As inscrições para a edição mais recente foram até o dia 11 de maio de 2025, e as aulas ocorreram entre 12 e 16 de maio de 2025.

No momento, não há informações sobre uma próxima edição da Imersão IA com Google Gemini. Para ficar por dentro de futuras edições, você pode:

*   Acessar o site da Alura e verificar a seção de imersões ou cursos de inteligência artificial.
*   Seguir a Alura nas redes sociais para receber atualizações sobre novos eventos e cursos.
*   Inscrever-se na newsletter da Alura para receber informações diretamente no seu e-mail.

In [6]:
# Exibe a busca
print(f"Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
# Exibe as URLs nas quais ele se baseou
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

Busca realizada: ['Alura Imersão IA Google Gemini', 'próxima Imersão IA com Google Gemini da Alura']
Páginas utilizadas na resposta: youtube.com, alura.com.br, starten.tech, alura.com.br, youtube.com



In [7]:
# Instalar Framework ADK de agentes do Google ################################################
!pip install -q google-adk

In [8]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [9]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [10]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [11]:
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################
def agente_buscador(topico, data_de_hoje):

    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        description = "Agente para buscar notícias no Google",
        tools=[google_search],
        instruction = """Você é um assistente de pesquisa, e sua tarefa é usar o Google Search para recuperar as últimas notícias de lançamentos muito relevantes sobre o tópico
        abaixo. Limite a 5 lançamentos relevantes, com base na quantidade e entusiasmo das notícias sobre eles.
        Se um tema tiver poucas notícias ou reações pouco entusiasmadas, é possível que ele não seja tão relevante, podendo ser substituído por outro mais relevante.
        Esse lançamentos relevantes devem ser atuais, com no máximo 1 mês da data de hoje."""
    )

    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"

    # Executa o agente
    lancamentos_buscados = call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos_buscados

In [12]:
################################################
# --- Agente 2: Planejador de posts --- #
################################################
def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction=""" Você é um planejador de conteúdo especialista em redes sociais. Com base na lista de lançamentos mais recentes e relevantes buscados,
        você deve usar o Google Search para criar um plano sobre quais são os pontos mais relevantes que poderíamos abordar em um post sobre cada um deles.
        Você também pode usar o Google Search para encontrar mais informações sobre os temas. Ao final, você escolherá o tema mais relevante com bases na suas pesquisa e retorná-lo,
        junto com seus pontos mais relevantes, e um plano com os assuntos a serem abordados no post que será escrito posteriormente.
        """,
        description="Agente que planeja posts",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [13]:
######################################
# --- Agente 3: Redator do Post --- #
######################################
def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Redator Criativo especializado em criar posts virais para redes sociais.
            Você escreve posts para a empresa Alura, a maior escola online de tecnologia do Brasil.
            Utilize o tema fornecido no plano de post e os pontos mais relevantes fornecidos e, com base nisso,
            escreva um rascunho de post para Instagram sobre o tema indicado.
            O post deve ser engajador, informativo, com linguagem simples e incluir 2 a 4 hashtags no final.
            """,
        description="Agente redator de posts engajadores para Instagram"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    # Executa o agente
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

In [14]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em posts para redes sociais, com foco no Instagram.
            Por ter um público jovem, entre 18 e 30 anos, use um tom de escrita adequado.
            Revise o rascunho de post de Instagram abaixo sobre o tópico indicado, verificando clareza, concisão, correção e tom.
            Se o rascunho estiver bom, responda apenas 'O rascunho está ótimo e pronto para publicar!'.
            Caso haja problemas, aponte-os e sugira melhorias.
            """,
        description="Agente revisor de post para redes sociais."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [15]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: ")

# Inserir lógica do sistema de agentes ################################################
while not topico:
  print("Por favor, digite o TÓPICO!!")
  topico = input()

print(f"Criando o post sobre {topico}....")

lancamentos_buscados = agente_buscador(topico, data_de_hoje)
print("\n--- Resultados ---\n")
display(to_markdown(lancamentos_buscados))

🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀
❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: engenharia mecatrônica em recife
Criando o post sobre engenharia mecatrônica em recife....

--- Resultados ---



> Para encontrar os lançamentos mais relevantes na área de engenharia mecatrônica em Recife, considerando notícias recentes e entusiasmo em torno dos eventos, vou realizar algumas buscas no Google.
> 
> 
> Com base nas minhas buscas, aqui estão alguns lançamentos e eventos relevantes em engenharia mecatrônica em Recife e arredores, com um foco em notícias e oportunidades recentes:
> 
> 1.  **SENAI Pernambuco:** O SENAI oferece diversos cursos técnicos em áreas como Automação Industrial, Eletrotécnica, Mecânica e Mecatrônica, tanto presenciais quanto a distância. Recentemente, o SENAI tem aberto editais com vagas gratuitas para cursos técnicos, sendo uma excelente oportunidade para quem busca qualificação profissional. Fique de olho no site do SENAI para os próximos editais.
> 
> 2.  **FIAP em Recife:** A FIAP oferece o curso de Engenharia Mecatrônica – Robotics Systems & Machine Learning. O curso tem uma grade curricular voltada para a Indústria 4.0, com foco em robótica, automação e inteligência artificial. A FIAP também oferece o programa Startup One, que permite aos alunos criarem suas próprias startups na área de mecatrônica como projeto de conclusão de curso. O vestibular para ingresso em agosto de 2025 está com inscrições abertas.
> 
> 3.  **Congresso Nacional de Ciência, Tecnologia e Engenharia de Alimentos (CONALI 2025):** Embora o foco principal seja em Engenharia de Alimentos, o evento, que será realizado em Recife, inclui uma área temática sobre Automação, Engenharia e Instalações industriais, o que pode ser relevante para engenheiros mecatrônicos que atuam na indústria de alimentos. O congresso é uma oportunidade para troca de conhecimentos e apresentação de trabalhos científicos.
> 
> 4.  **Siemens Tech Tour em Recife (2024):** Embora este evento tenha ocorrido em maio de 2024, a Siemens realizou um Tech Tour em Recife com foco em soluções de Indústria Digital e Automação. A Siemens apresentou as mais recentes soluções para Automação de Processos, Automação Industrial e Motion Control, além de produtos elétricos de baixa tensão. É importante ficar atento aos futuros eventos da Siemens na região.
> 
> 5.  **Laboratório de Manufatura Avançada (LMA/IIT) na UPE:** O Laboratório de Manufatura Avançada da Universidade de Pernambuco (UPE) oferece suporte a atividades relacionadas à manufatura, incluindo projetos de pesquisa, desenvolvimento e inovação. O laboratório abrange diversas áreas, incluindo Engenharia Mecatrônica, e oferece técnicas como projeto e prototipagem, usinagem CNC, visão computacional, manufatura aditiva e tratamento térmico.
> 
> Além dessas opções, vale a pena verificar os cursos de graduação e pós-graduação em Engenharia Mecatrônica oferecidos por outras universidades e instituições de ensino em Recife, como a Universidade Federal de Pernambuco (UFPE) e o Instituto Federal de Pernambuco (IFPE).


In [17]:
print("\n-----------------------\n")
plano_de_post = agente_planejador(topico, lancamentos_buscados)
print("\n--- Resultados ---\n")
display(to_markdown(plano_de_post))

print("\n-----------------------\n")
rascunho_de_post = agente_redator(topico, plano_de_post)
print("\n--- Resultados ---\n")
display(to_markdown(rascunho_de_post))

print("\n-----------------------\n")
post_final = agente_revisor(topico, rascunho_de_post)
print("\n--- Resultados ---\n")
display(to_markdown(post_final))



-----------------------


--- Resultados ---



> Com base nos lançamentos e eventos relevantes em engenharia mecatrônica em Recife, aqui está um plano de conteúdo para redes sociais, destacando os pontos mais relevantes e um plano com os assuntos a serem abordados em um post:
> 
> **Tópico Escolhido:** FIAP em Recife: Engenharia Mecatrônica – Robotics Systems & Machine Learning
> 
> **Relevância:**
> 
> *   Grade curricular voltada para a Indústria 4.0, um tema de grande interesse e atualidade.
> *   Foco em robótica, automação e inteligência artificial, áreas de alta demanda no mercado.
> *   Programa Startup One, que incentiva o empreendedorismo e a criação de startups na área de mecatrônica.
> *   Vestibular aberto para ingresso em agosto de 2025, um atrativo para potenciais alunos.
> 
> **Pontos Relevantes:**
> 
> 1.  **Indústria 4.0:** Explique como a grade curricular do curso prepara os alunos para os desafios e oportunidades da Indústria 4.0.
> 2.  **Robótica, Automação e Inteligência Artificial:** Destaque a importância dessas áreas no mercado de trabalho e como o curso oferece uma formação completa nesses campos.
> 3.  **Programa Startup One:** Incentive o empreendedorismo, mostrando como o programa permite aos alunos criarem suas próprias startups como projeto de conclusão de curso.
> 4.  **Vestibular Aberto:** Informe sobre o vestibular para ingresso em agosto de 2025 e como os interessados podem se inscrever.
> 
> **Plano do Post:**
> 
> 1.  **Título:** "Engenharia Mecatrônica na FIAP Recife: Prepare-se para a Indústria 4.0 e Crie sua Startup!"
> 2.  **Introdução:**
>     *   Apresente a FIAP como referência em tecnologia e inovação.
>     *   Destaque a importância da Engenharia Mecatrônica na Indústria 4.0.
> 3.  **Corpo do Post:**
>     *   Explique como o curso de Engenharia Mecatrônica da FIAP prepara os alunos para os desafios e oportunidades da Indústria 4.0.
>     *   Destaque o foco em robótica, automação e inteligência artificial.
>     *   Apresente o programa Startup One e como ele incentiva o empreendedorismo.
>     *   Informe sobre o vestibular aberto para ingresso em agosto de 2025.
> 4.  **Chamada para Ação:**
>     *   Incentive os interessados a se inscreverem no vestibular.
>     *   Convide os leitores a conhecerem mais sobre o curso no site da FIAP.
> 5.  **Hashtags:**
>     *   #EngenhariaMecatronica #FIAPRecife #Industria40 #Robotica #Automacao #InteligenciaArtificial #StartupOne #VestibularAberto
> 
> 
> Com base nas informações adicionais coletadas, o plano de conteúdo foi atualizado com mais detalhes e informações relevantes para o público:
> 
> **Tópico Escolhido:** FIAP em Recife: Engenharia Mecatrônica – Robotics Systems & Machine Learning
> 
> **Relevância:**
> 
> *   Curso alinhado com a Indústria 4.0, preparando profissionais para a robótica, automação e inteligência artificial.
> *   Programa Startup One que incentiva o desenvolvimento de startups como projeto de conclusão.
> *   Vestibular aberto para agosto de 2025.
> 
> **Pontos Relevantes:**
> 
> 1.  **Indústria 4.0:**
>     *   O curso prepara os alunos para a 4ª Revolução Industrial, onde robótica, automação e mecatrônica são catalisadores de mudanças.
>     *   Foco em sistemas automatizados, utilizando tecnologias exponenciais para manter a competitividade.
> 2.  **Robótica, Automação e Inteligência Artificial:**
>     *   O curso oferece aprendizado prático voltado à criação e implementação de produtos e processos inteligentes inéditos em engenharia.
>     *   Ênfase em Inteligência Artificial, Robótica, Sensores, Design, Sistemas Embarcados e Energias Alternativas.
> 3.  **Programa Startup One:**
>     *   Incentiva os alunos a criarem suas próprias startups na área da Mecatrônica como projeto de conclusão de curso.
>     *   Transforma digitalmente negócios, aliando empreendedorismo, criatividade e Design Thinking.
> 4.  **Vestibular Aberto:**
>     *   Inscrições abertas até 31 de julho de 2025 para o vestibular digital em 2 de agosto de 2025.
>     *   Informações sobre o vestibular e inscrições podem ser encontradas no site da FIAP.
> 
> **Plano do Post:**
> 
> 1.  **Título:** "Engenharia Mecatrônica na FIAP Recife: Prepare-se para a Indústria 4.0 e Crie sua Startup!"
> 2.  **Introdução:**
>     *   Apresente a FIAP como referência em tecnologia e inovação.
>     *   Destaque a importância da Engenharia Mecatrônica na Indústria 4.0 e o papel crucial da robótica, automação e inteligência artificial.
> 3.  **Corpo do Post:**
>     *   Explique como o curso prepara os alunos para os desafios e oportunidades da Indústria 4.0, com ênfase nas áreas de robótica, automação e inteligência artificial.
>     *   Apresente o programa Startup One, incentivando o empreendedorismo e destacando como os alunos desenvolvem suas próprias startups.
>     *   Mencione as tecnologias e metodologias disruptivas abordadas, como Nanotecnologia, Robôs Autônomos, Sensores, Inteligência Artificial e Energias Alternativas.
>     *   Informe sobre o vestibular aberto para ingresso em agosto de 2025, com inscrições até 31 de julho, e direcione para o site da FIAP para mais informações e inscrições.
> 4.  **Chamada para Ação:**
>     *   Incentive os interessados a se inscreverem no vestibular.
>     *   Convide os leitores a conhecerem mais sobre o curso e a FIAP no site oficial.
> 5.  **Hashtags:**
>     *   #EngenhariaMecatronica #FIAPRecife #Industria40 #Robotica #Automacao #InteligenciaArtificial #StartupOne #VestibularAberto #Inovacao #Tecnologia
> 



-----------------------


--- Resultados ---



> 🚀 Engenharia Mecatrônica na FIAP Recife: A CHAVE para a Indústria 4.0! 🤖
> 
> A FIAP, referência em tecnologia e inovação, te prepara para o futuro! 🤩 Nosso curso de Engenharia Mecatrônica te capacita para dominar a Indústria 4.0, com foco em robótica, automação e inteligência artificial. 
> 
> 💡 Imagine criar soluções inovadoras e ainda ter a chance de lançar sua própria startup com o Programa Startup One! 
> 
> 🎯 No curso, você vai mergulhar em:
> *   Robótica
> *   Automação
> *   Inteligência Artificial
> *   Sensores
> *   Sistemas Embarcados
> *   Energias Alternativas
> e muito mais!
> 
> 🗓️ As inscrições para o vestibular estão abertas até 31 de julho de 2025! Não perca essa chance de transformar o seu futuro.
> 
> 👉 Inscreva-se agora mesmo e saiba mais sobre o curso no site da FIAP! Link na bio. 😉
> 
> #EngenhariaMecatronica #FIAPRecife #Industria40 #Robotica #Automacao #InteligenciaArtificial #StartupOne #VestibularAberto
> 



-----------------------


--- Resultados ---



> O rascunho está ótimo, mas podemos fazer alguns ajustes para deixá-lo ainda mais atraente e direto para o público do Instagram:
> 
> *   **Emojis:** Use emojis com moderação, eles são bons para chamar a atenção, mas em excesso podem distrair.
> *   **Chamada para Ação:** Seja mais direto na chamada para ação. Incentive o clique no link da bio de forma mais clara.
> *   **Hashtags:** As hashtags estão boas, mas tente mesclar algumas mais específicas com outras mais genéricas para aumentar o alcance.
> 
> **Sugestão de Rascunho Revisado:**
> 
> 🚀 DECOLA NA INDÚSTRIA 4.0 COM A ENGENHARIA MECATRÔNICA DA FIAP RECIFE! 🤖
> 
> Quer dominar as tecnologias do futuro e ainda ter a chance de criar sua própria startup? 🤩 Na FIAP Recife, você aprende robótica, automação, inteligência artificial e muito mais, com foco total na Indústria 4.0.
> 
> 💡 No curso, você vai mergulhar em:
> 
> *   Robótica
> *   Automação
> *   Inteligência Artificial
> *   Sensores
> *   Sistemas Embarcados
> *   Energias Alternativas
> 
> 🔥 E tem mais: com o Programa Startup One, você pode transformar sua ideia em realidade!
> 
> 🗓️ Inscrições ABERTAS para o vestibular até 31 de julho de 2025!
> 
> 👉 Curtiu? Então corre pro link da bio e GARANTA SUA VAGA! 😉
> 
> #EngenhariaMecatronica #FIAPRecife #Industria40 #Robotica #Automacao #InteligenciaArtificial #StartupOne #VestibularAberto #TecnologiaRecife #Inovacao
> 
> **Justificativas das mudanças:**
> 
> *   **Título:** Mais direto e chamativo.
> *   **Chamada para Ação:** Mais enfática ("GARANTA SUA VAGA!").
> *   **Hashtags:** Adicionadas hashtags mais específicas para o público local ("#TecnologiaRecife", "#Inovacao").
> *   **Remoção de um emoji:** Para não poluir visualmente o post.
